In [68]:
import cv2
import sys
import os
import dlib
import glob
import csv
from skimage import io
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
%matplotlib inline

In [69]:
images = []
labels = []
resize_x = 80
resize_y = 80

In [70]:
def conv2d(x, W):
    return tf.nn.conv2d(
        x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(
        x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [71]:
with open('../data/convolutional/data.csv', 'rb') as mycsvfile:
    thedata = csv.reader(mycsvfile, delimiter=':')
    thedata.next()
    for row in thedata:
        images.append(np.fromstring(row[0].replace("[", "").replace("]", ""), dtype=int, sep=" "))
        labels.append(row[1])

In [72]:
labels = np.array(labels)

In [73]:
df_labels = pd.get_dummies(labels)
labels = np.array(df_labels)

In [74]:
images = np.array(images) / 255

In [75]:
X_train = images[:200]
Y_train = labels.reshape(len(labels), -1)[:200]

X_test = images[200:]
Y_test = labels.reshape(len(labels), -1)[200:]

In [76]:
# Input and output variables

INPUTS = resize_x * resize_y * 3
OUTPUTS = 2
BATCH_SIZE = 20
NUM_EPOCHS = 51
LEARNING_RATE = 1e-04

In [77]:
# Interactive session because of notebook environment
sess = tf.InteractiveSession()

# Input and output placeholder
x = tf.placeholder(tf.float32, [None, INPUTS])
y = tf.placeholder(tf.float32, [None, OUTPUTS])

# First Convolutional Layer
# Input and output placeholder
x = tf.placeholder(tf.float32, [None, INPUTS])
y = tf.placeholder(tf.float32, [None, OUTPUTS])
pkeep = tf.placeholder(tf.float32)

# First Convolutional Layer
x_image = tf.reshape(x, [-1, resize_x, resize_y, 3])
W_conv_1 = tf.Variable(tf.truncated_normal([2, 2, 3, 8], stddev=0.1))
b_conv_1 = tf.Variable(tf.constant(0.0, shape=[8]))
h_conv_1 = tf.nn.relu(conv2d(x_image, W_conv_1) + b_conv_1)
h_pool_1 = max_pool_2x2(h_conv_1)

# Second Convolutional Layer
W_conv_2 = tf.Variable(tf.truncated_normal([2, 2, 8, 32], stddev=0.1))
b_conv_2 = tf.Variable(tf.constant(0.0, shape=[32]))
h_conv_2 = tf.nn.relu(conv2d(h_pool_1, W_conv_2) + b_conv_2)
h_pool_2 = max_pool_2x2(h_conv_2)

# Third Convolutional Layer
W_conv_3 = tf.Variable(tf.truncated_normal([2, 2, 32, 128], stddev=0.1))
b_conv_3 = tf.Variable(tf.constant(0.0, shape=[128]))
h_conv_3 = tf.nn.relu(conv2d(h_pool_2, W_conv_3) + b_conv_3)
h_pool_3 = max_pool_2x2(h_conv_3)

# Fourth Convolutional Layer
W_conv_4 = tf.Variable(tf.truncated_normal([2, 2, 128, 256], stddev=0.1))
b_conv_4 = tf.Variable(tf.constant(0.0, shape=[256]))
h_conv_4 = tf.nn.relu(conv2d(h_pool_3, W_conv_4) + b_conv_4)
h_pool_4 = max_pool_2x2(h_conv_4)

# Densely connected layer
W_fc1 = tf.Variable(tf.truncated_normal([5 * 5 * 256, 1024], stddev=0.1))
b_fc1 = tf.Variable(tf.constant(0.0, shape=[1024]))
h_poolfc1_flat = tf.reshape(h_pool_4, [-1, 5 * 5 * 256])
h_fc1 = tf.nn.relu(tf.matmul(h_poolfc1_flat, W_fc1) + b_fc1)

# Densely connected layer
W_fc2 = tf.Variable(tf.truncated_normal([1024, 512], stddev=0.1))
b_fc2 = tf.Variable(tf.constant(0.0, shape=[512]))
h_fc2 = tf.nn.relu(tf.matmul(h_fc1, W_fc2) + b_fc2)

# Dropout
h_drop = tf.nn.dropout(h_fc2, pkeep)

# Read out Layer
W_fc3 = tf.Variable(tf.truncated_normal([512, OUTPUTS], stddev=0.1))
b_fc3 = tf.Variable(tf.constant(0.0, shape=[OUTPUTS]))

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(tf.matmul(h_drop, W_fc3) + b_fc3, y))
optimizer = tf.train.AdamOptimizer(LEARNING_RATE)
train_step = optimizer.minimize(loss)

correct_prediction = tf.equal(tf.argmax(tf.matmul(h_drop, W_fc3) + b_fc3, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

init = tf.initialize_all_variables()
sess.run(init)

Exception AssertionError: AssertionError("Nesting violated for default stack of <type 'weakref'> objects",) in <bound method InteractiveSession.__del__ of <tensorflow.python.client.session.InteractiveSession object at 0x7f95d05c6750>> ignored


In [78]:
loss_train_array = []
test_accuracy_array = []
train_accuracy_array = []

for current_epoch in range(NUM_EPOCHS):
    shuffled_index = np.arange(X_train.shape[0])
    np.random.shuffle(shuffled_index)

    train_dataset = X_train[shuffled_index]
    train_labels = Y_train[shuffled_index]

    for step in xrange(int(X_train.shape[0] / BATCH_SIZE)):
        
        offset = step * BATCH_SIZE
        batch_data = train_dataset[offset:(offset + BATCH_SIZE)]
        batch_labels = train_labels[offset:(offset + BATCH_SIZE)]
        
        # This dictionary maps the batch data (as a numpy array) to the
        # node in the graph is should be fed to.
        feed_dict = {x: batch_data, y: batch_labels, pkeep: 0.8}
        _, loss_train = sess.run([train_step, loss],
                                 feed_dict=feed_dict)

    # We calculate the accuracies to plot their values later
    loss_train_array.append(loss_train)
    
    train_accuracy = sess.run(
        accuracy, feed_dict={x: X_train, y: Y_train, pkeep: 1.0})
    
    train_accuracy_array.append(train_accuracy)
    
    test_accuracy = sess.run(
        accuracy, feed_dict={x: X_test, y: Y_test, pkeep: 1.0})
    
    test_accuracy_array.append(test_accuracy)

    print (
        'Epoch %04d, '
        'loss train %.8f, train accuracy %.8f, test accuracy %.8f'
        %
        (current_epoch,
         loss_train,
         train_accuracy,
         test_accuracy))

Epoch 0000, loss train 0.00015511, train accuracy 1.00000000, test accuracy 0.02242152
Epoch 0001, loss train 0.00001295, train accuracy 1.00000000, test accuracy 0.02242152
Epoch 0002, loss train 0.00000163, train accuracy 1.00000000, test accuracy 0.02242152
Epoch 0003, loss train 0.00000147, train accuracy 1.00000000, test accuracy 0.02242152
Epoch 0004, loss train 0.00000174, train accuracy 1.00000000, test accuracy 0.02242152
Epoch 0005, loss train 0.00000080, train accuracy 1.00000000, test accuracy 0.02242152
Epoch 0006, loss train 0.00000096, train accuracy 1.00000000, test accuracy 0.02242152
Epoch 0007, loss train 0.00000127, train accuracy 1.00000000, test accuracy 0.02242152
Epoch 0008, loss train 0.00000071, train accuracy 1.00000000, test accuracy 0.02242152
Epoch 0009, loss train 0.00000088, train accuracy 1.00000000, test accuracy 0.02242152
Epoch 0010, loss train 0.00000113, train accuracy 1.00000000, test accuracy 0.02242152
Epoch 0011, loss train 0.00000170, train ac

In [79]:
# Comparing values
y_ = tf.matmul(h_drop, W_fc3) + b_fc3
predicted = y_.eval(feed_dict={x: X_train, pkeep: 0.8})
predicted

array([[  8.22127724,  -9.49939919],
       [  9.40658951,  -9.42617607],
       [  9.24275208,  -8.16240883],
       [  9.09773445,  -9.10353947],
       [  9.20250702,  -8.76465034],
       [ 10.35251713,  -9.68080521],
       [ 12.78678894, -10.7579031 ],
       [  8.62435722,  -8.69153118],
       [  9.81079865,  -9.34579659],
       [  8.86642456,  -6.97173214],
       [  7.82815504,  -8.048769  ],
       [  9.57257843,  -9.2912693 ],
       [  9.59804726,  -8.50889206],
       [ 14.82345009,  -9.55526257],
       [  9.16401482,  -8.50673485],
       [  8.84998894,  -8.21933174],
       [  8.40625191, -10.09387016],
       [  8.87727356,  -8.79297066],
       [  7.30653191,  -7.17975569],
       [  8.14518547,  -7.80852985],
       [  8.3234911 ,  -8.81930542],
       [  8.68849373,  -7.64343834],
       [  8.98505974,  -8.68725586],
       [  8.66062546, -10.0037384 ],
       [  8.20461082,  -7.81984091],
       [  8.01020622,  -8.83095932],
       [ 18.16669273, -14.4156208 ],
 